In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))


from src.data_processing import run_data_processing

In [2]:
processed_df = run_data_processing(
raw_path="../data/raw/data.csv",
output_path="../data/processed/proccessed.csv",
target="FraudResult",
use_woe=True, # Set True if WoE is required
scale_method="standard" # or "minmax"
)

In [3]:
processed_df.head()


,0,1,2,3,4,5,6,7,8,9,...,49,50,51,52,53,54,55,56,57,FraudResult
0,0.0,-0.046371,-0.072291,-2.155530,-0.100739,0.848684,-0.994246,0.170118,-0.067623,-0.311831,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0
1,0.0,-0.054643,-0.080251,-2.155530,-0.100739,0.848684,-0.994246,0.170118,-0.067623,-0.311831,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0
2,0.0,-0.050426,-0.076352,-2.155530,-0.100739,0.848684,-0.994246,0.165122,-0.072568,-0.444993,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0
3,0.0,0.107717,0.096648,-1.949214,-0.100739,0.848684,-0.994246,0.175567,-0.008155,-0.404020,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0
4,0.0,-0.059704,-0.075183,-1.949214,-0.100739,0.848684,-0.994246,0.175567,-0.008155,-0.404020,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0


In [4]:
processed_df.shape

(95662, 59)